In [1]:
import torch
import yaml

# Local imports
import carl_utils.preprocessing as carl_ppro
import carl_utils.train as carl_train

import logging
logging.getLogger('matplotlib.font_manager').setLevel(logging.ERROR)

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#DEVICE = 'cpu'
print(DEVICE)

cuda:0


In [2]:
total_features = yaml.load(open("carl_features.yaml", 'r'), Loader=yaml.CLoader)
total_features

{'Electron_set': {'set': True,
  'size': 4,
  'subfeatures': ['Electron_pt',
   'Electron_eta',
   'Electron_phi',
   'Electron_mass']},
 'FatJet_set': {'set': True,
  'size': 5,
  'subfeatures': ['FatJet_pt',
   'FatJet_eta',
   'FatJet_phi',
   'FatJet_mass',
   'FatJet_btagCSVV2']},
 'Jet_set': {'set': True,
  'size': 5,
  'subfeatures': ['Jet_pt',
   'Jet_eta',
   'Jet_phi',
   'Jet_mass',
   'Jet_btagCSVV2']},
 'MET_sumEt': {'set': False, 'size': 1, 'subfeatures': ['MET_sumEt']},
 'Muon_set': {'set': True,
  'size': 4,
  'subfeatures': ['Muon_pt', 'Muon_eta', 'Muon_phi', 'Muon_mass']},
 'Photon_set': {'set': True,
  'size': 4,
  'subfeatures': ['Photon_pt', 'Photon_eta', 'Photon_phi', 'Photon_mass']}}

In [3]:
subfeatures = ["Jet_set", "MET_sumEt", "Electron_set", "FatJet_set"]
features = dict(zip(subfeatures, [total_features[feat] for feat in subfeatures]))
features

{'Jet_set': {'set': True,
  'size': 5,
  'subfeatures': ['Jet_pt',
   'Jet_eta',
   'Jet_phi',
   'Jet_mass',
   'Jet_btagCSVV2']},
 'MET_sumEt': {'set': False, 'size': 1, 'subfeatures': ['MET_sumEt']},
 'Electron_set': {'set': True,
  'size': 4,
  'subfeatures': ['Electron_pt',
   'Electron_eta',
   'Electron_phi',
   'Electron_mass']},
 'FatJet_set': {'set': True,
  'size': 5,
  'subfeatures': ['FatJet_pt',
   'FatJet_eta',
   'FatJet_phi',
   'FatJet_mass',
   'FatJet_btagCSVV2']}}

In [4]:
weight_features = ["genWeight", "btagWeight_CSVV2"]

## Initiate the datasets and load the data

In [5]:
n_train_events = int(400000) // 2
n_val_events = int(1e5) // 2

train_nominal_dataset = carl_ppro.DeepSetsDataset(["/data/mdrnevich/AGC/CMS_ttbar_nominal_DeepSets_training_data_new2.root",], features, 0, stop_event=n_train_events)
valid_nominal_dataset = carl_ppro.DeepSetsDataset(["/data/mdrnevich/AGC/CMS_ttbar_nominal_DeepSets_validation_data_new2.root",], features, 0, stop_event=n_val_events)

train_PS_var_dataset = carl_ppro.DeepSetsDataset(["/data/mdrnevich/AGC/CMS_ttbar_PS_var_DeepSets_training_data_new2.root",], features, 1, stop_event=n_train_events)
valid_PS_var_dataset = carl_ppro.DeepSetsDataset(["/data/mdrnevich/AGC/CMS_ttbar_PS_var_DeepSets_validation_data_new2.root",], features, 1, stop_event=n_val_events)

100%|██████████| 50000/50000 [00:23<00:00, 2102.38it/s]


## Prepare the data

In [6]:
train_generator_data = carl_ppro.CombinedDataset(train_nominal_dataset, train_PS_var_dataset)
valid_generator_data = carl_ppro.CombinedDataset(valid_nominal_dataset, valid_PS_var_dataset)

## Define the settings of the model and training

In [7]:
hidden1 = 512
hidden2 = 256
hidden3 = 128
classify1 = 1024
classify2 = 1024
outputs = 1

model_settings = {
    "features": features,
    "phi": [hidden1, hidden2, hidden3],
    "mlp": [classify1, classify2]
}


training_settings = {
    "optimizer": "Adam",
    "learning_rate": 1e-2,
    "batch_size": 1024,
    "n_epochs": 30,
    "patience": 6,
    "device": DEVICE,
    "n_events": n_train_events * 2
}

# Perform the training

In [8]:
model = carl_train.train(model_settings, train_generator_data, valid_generator_data, **training_settings)

Constructing the model
Loading the input data scaling


100%|██████████| 391/391 [00:06<00:00, 58.11it/s]


Training the model


  3%|▎         | 1/30 [03:50<1:51:29, 230.66s/it]                

Epoch: 01, Training Loss:   0.7038
           Validation Loss: 0.6872
New best model saved to: deepsets_model.zip


...

 63%|██████▎   | 19/30 [1:09:47<40:01, 218.31s/it]               

Epoch: 19, Training Loss:   0.6794
           Validation Loss: 0.6816
Stale epoch


 67%|██████▋   | 20/30 [1:13:28<36:31, 219.11s/it]               

Epoch: 20, Training Loss:   0.6793
           Validation Loss: 0.6821
Stale epoch


Epoch: 21, Training Loss:   0.6790
           Validation Loss: 0.6820
Stale epoch
Early stopping after 6 stale epochs
Finished training
